## Cleaning coal data:
    Starting with files from from https://globalenergymonitor.org/projects/global-coal-plant-tracker/summary-data/ - retrieved 2021-03-11. Files are coal power new and retired capacity in Mega Watts (MW or mw)

In [1]:
#import dependencies
import pandas as pd #dataframe
import numpy as np  #array methods, numeric fns

In [2]:
#load source files as dataframes
path_new = 'data/new_coal_MW.csv'
path_retired = 'data/retired_coal_MW.csv'
path_lkp = 'data/country_lookup.csv'

new = pd.read_csv(path_new, header=3) #ignore the top 3 rows (zero index). headers are in row 4
retired = pd.read_csv(path_retired, header=3)
lkp = pd.read_csv(path_lkp)

In [3]:
#removing subtotal rows and total columns found after inspecting files
new.drop('Total 2000-2020', axis=1, inplace = True) #totals column
new.drop(range(108,112), axis=0, inplace = True) #subtotal rows

cols_to_drop = ['Dated 2000-2020','All retired (includes dated and undated)']

retired.drop(cols_to_drop, axis=1, inplace = True) # subtotal and totals columns
retired.drop(range(108,111), axis=0, inplace = True) #subtotal rows


### Issue: 
    material amount of retired capacity is in 'undated' column. 
    After discussion with team the consensus decision is that 
    the undated should all be rolled into 2020 numbers, since 
    we know it was retired by 2020 at the latest. 
    
    _Sub-issue:_ 
    numeric values are stored as string 
    with comma for thousands separator. 
    Removing so that I can convert the values to integer


In [4]:
#retired:
cols_to_convert = retired.columns[1:] #col zero is country name, so it is not to be converted
for each_col in cols_to_convert:
    retired[each_col]=retired[each_col].str.replace(',','').astype(int)

    
#new:
cols_to_convert = new.columns[1:] #col zero is country name, so it is not to be converted
for each_col in cols_to_convert:
    new[each_col]=new[each_col].str.replace(',','').astype(int)


In [5]:
#Roll 'undated' column for retired into year 2020
retired['2020_adj'] = retired['2020']+retired['Undated']


In [6]:
#Remove undated and original 2020 columns:
cols_to_drop = ['Undated','2020']

retired.drop(cols_to_drop, axis=1, inplace = True)

In [7]:
#rename 2020_adj to 2020 so all year labels may be treated as integers later
#This will allow for aggregation across years
#wrap Country column name to lowercase for consistency
colrename = {
    '2020_adj':'2020'
    ,'Country': 'country'
}
retired.rename(columns=colrename, inplace = True)

In [8]:
#change column name in new df as well:
new.rename(columns={'Country': 'country'}, inplace=True)

# reminder: 
later will unpivot data such that individual years are all in the same column. At that point, year numbers should be converted to integer.

### Issue: 
    country name needed to pull in three letter ISO country 
    code (alpha_3) from country_lookup.csv as Foreign Key, 
    but country names are not standardized. 
    Comparing country names across files:

In [9]:
new.head()

,country,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Albania,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Argentina,0,0,0,0,0,0,0,0,0,...,0,0,0,0,120,0,0,0,0,0
2,Australia,0,1071,440,885,0,0,0,750,0,...,0,144,0,0,0,0,0,0,0,0
3,Austria,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Bangladesh,0,0,0,0,0,0,250,0,0,...,0,0,0,0,0,0,0,275,0,660


In [10]:
#Checking that new and retired have the same countries:
col_to_check = 'country'

s1 = new[col_to_check]
s2 = retired[col_to_check]

In [11]:
  #had to upgrade pandas, was at 1.0.5, need 1.1.0 or higher... ended up on 1.2.3
if len(s1.compare(s2))==0:
    print('country names identical in new vs retired')

country names identical in new vs retired


In [12]:
#Need a new dataframe with just the country names and alpha_3 from country_lookup
cols_to_keep = ['name', 'alpha_3']
codes = lkp[cols_to_keep]
codes.head(10)

,name,alpha_3
0,Afghanistan,AFG
1,Aland Islands,ALA
2,Albania,ALB
3,Algeria,DZA
4,American Samoa,ASM
5,Andorra,AND
6,Angola,AGO
7,Anguilla,AIA
8,Antarctica,ATA
9,Antigua and Barbuda,ATG


In [13]:
source = s1.to_frame(name = 'coal_country') #convert series to df so I can compare using a left join
source_to_code = source.merge(codes, how='left', left_on='coal_country', right_on='name', indicator = True)

In [14]:
#checking for any countries that did not match up
if len(source_to_code['_merge'].unique()) >1 : # I know at least some country names matched up. So if the _merge field has more than one value, not all matched.
    print("Mismatches found")

Mismatches found


In [15]:
is_miss = source_to_code["_merge"] == "left_only"  #find those that didn't match
print(source_to_code[is_miss])

                    coal_country name alpha_3     _merge
7         Bosnia and Herzegovina  NaN     NaN  left_only
19  Democratic Republic of Congo  NaN     NaN  left_only
43                   Ivory Coast  NaN     NaN  left_only
49                        Kosovo  NaN     NaN  left_only
69                   North Korea  NaN     NaN  left_only
70               North Macedonia  NaN     NaN  left_only


In [16]:
#manually id correct spellings as defined by country_lookup.csv
print(codes[codes['name'].str.startswith('Bos')]) #note: OR operator does not appear to work when passed as part of arg to str.startswith()
print(codes[codes['name'].str.startswith('Co')])
print(codes[codes['name'].str.startswith('Korea')]) 
print(codes[codes['name'].str.startswith('Macedonia')])
print(codes[codes['name'].str.startswith('Kos')]) 


                      name alpha_3
28  Bosnia And Herzegovina     BIH
                                  name alpha_3
47             Cocos (Keeling) Islands     CCK
48                            Colombia     COL
49                             Comoros     COM
50                               Congo     COG
51  Congo (Democratic Republic Of The)     COD
52                        Cook Islands     COK
53                          Costa Rica     CRI
54                       Cote d'Ivoire     CIV
                   name alpha_3
119  Korea, Republic of     KOR
          name alpha_3
164  Macedonia     MKD
Empty DataFrame
Columns: [name, alpha_3]
Index: []


In [17]:
#correct spelling mismatches via dictionary passed to series.map()
new_spelling = {
    'Bosnia and Herzegovina': 'Bosnia And Herzegovina'                    #capitalize And
    ,'Democratic Republic of Congo':'Congo (Democratic Republic Of The)'
    ,'Ivory Coast': "Cote d'Ivoire"
    ,'Kosovo': 'Kosovo'                                                    #Kosovo not in country_lookup.csv
    ,'North Korea': 'Korea, Republic of' 
    ,'North Macedonia': 'Macedonia'
}

In [18]:
df = source_to_code[['coal_country', 'alpha_3']]

In [19]:
s1 = df['coal_country']

In [20]:
s1.replace(new_spelling).iloc[[7, 19, 43, 49]]

7                 Bosnia And Herzegovina
19    Congo (Democratic Republic Of The)
43                         Cote d'Ivoire
49                                Kosovo
Name: coal_country, dtype: object

In [21]:
df['coal_country'].replace(new_spelling, inplace=True)

C:\Users\Test\anaconda3\lib\site-packages\pandas\core\series.py:4509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [22]:
df.iloc[[7, 19, 43, 49]] #now have the spellings corrected, bringing in alpha_3 field in its entirety

,coal_country,alpha_3
7,Bosnia And Herzegovina,NaN
19,Congo (Democratic Republic Of The),NaN
43,Cote d'Ivoire,NaN
49,Kosovo,NaN


In [23]:
source_to_code = df.merge(codes, how='left', left_on='coal_country', right_on='name', indicator = True)

In [24]:
source_to_code.iloc[[7, 19, 43, 49, 69, 70]]

,coal_country,alpha_3_x,name,alpha_3_y,_merge
7,Bosnia And Herzegovina,NaN,Bosnia And Herzegovina,BIH,both
19,Congo (Democratic Republic Of The),NaN,Congo (Democratic Republic Of The),COD,both
43,Cote d'Ivoire,NaN,Cote d'Ivoire,CIV,both
49,Kosovo,NaN,NaN,NaN,left_only
69,"Korea, Republic of",NaN,"Korea, Republic of",KOR,both
70,Macedonia,NaN,Macedonia,MKD,both


In [25]:
source_to_code.drop(['alpha_3_x'], axis=1, inplace=True)
source_to_code.drop(['_merge'], axis=1, inplace=True)

In [26]:
source_to_code.rename(columns={'alpha_3_y':'alpha_3'}, inplace=True)

In [27]:
#import alpha_3 field from result to source table, so that there is now alpha codes for the countries with alternative spelling
source['alpha_3'] = source_to_code['alpha_3'] 

In [28]:
source.iloc[[7, 19, 43, 49, 69, 70]] #Checking for alignment. 

,coal_country,alpha_3
7,Bosnia and Herzegovina,BIH
19,Democratic Republic of Congo,COD
43,Ivory Coast,CIV
49,Kosovo,NaN
69,North Korea,KOR
70,North Macedonia,MKD


In [29]:
#Result good, so repeating on data tables
new['alpha_3'] = source_to_code['alpha_3'] 
retired['alpha_3'] = source_to_code['alpha_3'] 

In [30]:
new.iloc[[7, 19, 43, 49, 69,70]] #verifying import

,country,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,alpha_3
7,Bosnia and Herzegovina,0,0,0,0,0,0,0,0,0,...,0,0,0,0,300,0,0,0,0,BIH
19,Democratic Republic of Congo,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,COD
43,Ivory Coast,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,CIV
49,Kosovo,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
69,North Korea,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,200,0,0,0,KOR
70,North Macedonia,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,MKD


In [31]:
#un-pivoting data into a normalizable format
new_vertical = new.melt(id_vars = ['country', 'alpha_3'], var_name='year', value_name='coal_mw_new')

In [32]:
new_vertical.head()

,country,alpha_3,year,coal_mw_new
0,Albania,ALB,2000,0
1,Argentina,ARG,2000,0
2,Australia,AUS,2000,0
3,Austria,AUT,2000,0
4,Bangladesh,BGD,2000,0


In [33]:
retired_vertical = retired.melt(id_vars = ['country', 'alpha_3'], var_name='year', value_name='coal_mw_retired')

In [34]:
#Note: Taking advantage of convenient accident that new and retired tables happen to have the same dimensions, members, sort orders (year then by country asc), and granularity
# This means the new and retired frames can be merged into one table with no data loss or corruption
df = new_vertical.copy()

In [35]:
df['coal_mw_retired'] = retired_vertical['coal_mw_retired']

In [36]:
df['coal_mw_change'] = df['coal_mw_new']-df['coal_mw_retired'] #calculating deltas over each intersection

In [37]:
df.describe()

,coal_mw_new,coal_mw_retired,coal_mw_change
count,2268.000000,2268.000000,2268.000000
mean,616.961640,168.885362,448.076279
std,4953.058887,1119.254441,4534.949691
min,0.000000,0.000000,-21673.000000
25%,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000
max,82291.000000,21723.000000,81505.000000


In [38]:
df.dtypes #checking data types

country            object
alpha_3            object
year               object
coal_mw_new         int32
coal_mw_retired     int32
coal_mw_change      int32
dtype: object

## Issue:
    year numbers stored as strings, need to convert to int

In [39]:
df['year'] = df['year'].astype(int)

In [40]:
df.dtypes

country            object
alpha_3            object
year                int32
coal_mw_new         int32
coal_mw_retired     int32
coal_mw_change      int32
dtype: object

In [41]:
#write to CSV for intake as a db table in postgres
out_file = 'data/coal_plants.csv'
df.to_csv(out_file)